<a href="https://colab.research.google.com/github/prashantgodhwani/CSE_587_DIC_Project/blob/master/Collaborative_Filtering_using_Surprise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Collaborative Filtering using Surprise**

In [ ]:
!pip install surprise 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3195807 sha256=259577454a0dd70e434ebb55cb1d61997f03647a27e6a60cd465f99ee8c53e10
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Libraries

In [ ]:
import pandas as pd
from surprise import *
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise import dataset

## Read Review and Business Dataset

In [ ]:
review=pd.read_csv('drive/MyDrive/DIC/review.csv')
business_final=pd.read_csv('drive/MyDrive/DIC/business_final.csv')
review = review.merge(business_final[['name','business_id']], how='inner')
review

,Unnamed: 0,review_id,user_id,business_id,stars,date,text,useful,funny,cool,sentiment_score,sentiment,super_score,name
0,91,bifEpGC-8TE19JPZwn-Rpw,_L2SZSwf7A6YSrIHy_q4cw,TIf6ZByLdAXratne18kjoA,5,2009-09-01,guys cater wedding ended different flavors cak...,2,0,0,0.9211,positive,0.98422,"""Craig's Cake Shop"""
1,93,BF0ANB54sc_f-3_howQBCg,ssuXFjkH4neiBgwv-oN4IA,JlNeaOymdVbE6_bubqjohg,1,2014-08-09,always go chevo chandler delicious one ahwatuk...,3,0,0,0.7964,positive,0.95928,"""Papa Chevo's Taco Shop"""
2,15312,-QgtOpsFzLHd58-Y1Ao2tA,BPKpLbR9NuWFAR9SUWpZOw,JlNeaOymdVbE6_bubqjohg,5,2014-06-29,great grilled chicken burrito believe coming c...,0,0,0,0.8481,positive,0.96962,"""Papa Chevo's Taco Shop"""
3,49074,zdhN3MBABBKi-9QHAh-G1A,sg5q7rz2_7PfaN-6JnLb5g,JlNeaOymdVbE6_bubqjohg,5,2014-05-01,everything far great sure reviews pleased food...,1,0,1,0.9463,positive,0.98926,"""Papa Chevo's Taco Shop"""
4,94,DbLUpPT61ykLTakknCF9CQ,ssuXFjkH4neiBgwv-oN4IA,0Rni7ocMC_Lg2UH0lDeKMQ,1,2014-08-09,place always dirty grimy twice back customer s...,6,0,0,-0.8481,negative,0.63038,"""Barro's Pizza"""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47273,99942,bH0lMpJQFjEPaD3ZkOK97w,mUbIcYFzkkvLXP0yV4RpqQ,_iUlt5rm-15QoLw0MEEg5A,5,2015-12-03,joe absolutely amazing love coming hard day wo...,0,0,0,0.8927,positive,0.97854,"""Mitchell's Restaurant Bar & Banquet Center"""
47274,99943,r1S9zlVzM5EyLOdRQIVA4Q,mUbIcYFzkkvLXP0yV4RpqQ,Fa2gj48WwaEPuKW_Lj9MwQ,1,2016-06-06,boyfriend always loved hooks recently ordering...,0,0,0,0.9694,positive,0.99388,"""Hook Fish & Chicken"""
47275,99952,icKT88mre2LCsR9jtjhDLw,CKRfBUqQGaVCYTKN5kDrzw,7V6udFB44qe3lg_a8jdVyg,4,2017-08-12,sugary treats cookie dough decadence simple ye...,1,0,1,0.9911,positive,0.99822,"""Doughlicious Yummy's"""
47276,99965,o882zQGOakne88rFRRyKWQ,CKRfBUqQGaVCYTKN5kDrzw,fhx1yUZlC1QwovIjdYeGiw,4,2017-04-24,friendly yummy affordable italian food tried b...,1,0,1,0.9738,positive,0.99476,"""Tavolo"""


## Define Scale

In [ ]:
reader = Reader(rating_scale=(0,1))

## Get the relevant Data

In [ ]:
data = Dataset.load_from_df(review[['user_id', 'business_id', 'super_score']], reader)

## Train, Test split

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
trainset = data.build_full_trainset()

## 1. KNN

### 1. KNNBaseline

In [ ]:
parameters={
    'name':['cosine','pearson'],
    'min_k': [3, 6, 9],
    'min_support': [True, False],
    'user_based':[True, False]
}

In [ ]:
grid=GridSearchCV(KNNBaseline, param_grid=parameters,n_jobs=-1)
grid.fit(data)

TypeError: ignored

In [ ]:
print(grid.best_score)
print(grid.best_params)

{'rmse': 0.0858236961945287, 'mae': 0.0542203818637799}
{'rmse': {'name': 'cosine', 'min_k': 9, 'min_support': True, 'user_based': True}, 'mae': {'name': 'cosine', 'min_k': 6, 'min_support': True, 'user_based': True}}


In [ ]:
algo = grid.best_estimator["rmse"]
algo.fit(data.build_full_trainset())

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


#### Make recommendations

In [ ]:
predictions = algo.test(testset)

#### Recommend 5 Restaurants to users

In [ ]:
user_1 = 'BPKpLbR9NuWFAR9SUWpZOw'
restaurants = review['business_id'].unique()
user_1_ratings = []
for restaurant in restaurants:
    user_1_ratings.append((restaurant, algo.predict(user_1, restaurant).est))

# sort the recommended restaurants by predicted rating
sorted_ratings = sorted(user_1_ratings, key=lambda x: x[1], reverse=True)

# print the top 5 recommended restaurants
for i in range(5):
    business_id = sorted_ratings[i][0]
    name = review.loc[review['business_id'] == business_id, 'name'].iloc[0]
    print(f'{i+1}. {name}')

1. "Yao Fine Chinese Cuisine"
2. "CUT by Wolfgang Puck"
3. "Scarpetta"
4. "Firefly"
5. "Oyster Bar"


In [ ]:
accuracy.mae(predictions)
accuracy.mse(predictions)
accuracy.rmse(predictions)
accuracy.fcp(predictions)

MAE:  0.0069
MSE: 0.0006
RMSE: 0.0240
FCP:  0.5600


0.5600329794908792

### 2. KNNWithMeans

In [ ]:
parameters={
    'name':['cosine','pearson'],
    'min_k': [3, 6, 9],
    'min_support': [True, False],
    'user_based':[True, False]
}

In [ ]:
grid=GridSearchCV(KNNWithMeans, param_grid=parameters,n_jobs=-1)
grid.fit(data)

In [ ]:
print(grid.best_score)
print(grid.best_params)

{'rmse': 0.0915131161471882, 'mae': 0.05605022367509525}
{'rmse': {'name': 'cosine', 'min_k': 9, 'min_support': True, 'user_based': True}, 'mae': {'name': 'cosine', 'min_k': 9, 'min_support': True, 'user_based': True}}


In [ ]:
algo = grid.best_estimator["rmse"]
algo.fit(data.build_full_trainset())

Computing the msd similarity matrix...
Done computing similarity matrix.


#### Make recommendations

In [ ]:
predictions = algo.test(testset)

#### Recommend 5 Restaurants to users

In [ ]:
user_1 = 'BPKpLbR9NuWFAR9SUWpZOw'
restaurants = review['business_id'].unique()
user_1_ratings = []
for restaurant in restaurants:
    user_1_ratings.append((restaurant, algo.predict(user_1, restaurant).est))

# sort the recommended restaurants by predicted rating
sorted_ratings = sorted(user_1_ratings, key=lambda x: x[1], reverse=True)

# print the top 5 recommended restaurants
for i in range(5):
    business_id = sorted_ratings[i][0]
    name = review.loc[review['business_id'] == business_id, 'name'].iloc[0]
    print(f'{i+1}. {name}')

1. "Papa Chevo's Taco Shop"
2. "Macayo's Mexican Table"
3. "Kona Grill"
4. "Joyride Taco House"
5. "Original Breakfast House"


In [ ]:
accuracy.mae(predictions)
accuracy.mse(predictions)
accuracy.rmse(predictions)
accuracy.fcp(predictions)

MAE:  0.0278
MSE: 0.0024
RMSE: 0.0492
FCP:  0.5522


0.55224913659326

### 3. KNN With ZScore

In [ ]:
parameters={
    'name':['cosine','pearson'],
    'min_k': [3, 6, 9],
    'min_support': [True, False],
    'user_based':[True, False]
}

In [ ]:
grid=GridSearchCV(KNNWithZScore, param_grid=parameters,n_jobs=-1)
grid.fit(data)

In [ ]:
print(grid.best_score)
print(grid.best_params)

{'rmse': 0.0919170838625821, 'mae': 0.05622203400847727}
{'rmse': {'name': 'cosine', 'min_k': 9, 'min_support': True, 'user_based': True}, 'mae': {'name': 'cosine', 'min_k': 6, 'min_support': True, 'user_based': True}}


In [ ]:
algo = grid.best_estimator["rmse"]
algo.fit(data.build_full_trainset())

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


#### Make recommendations

In [ ]:
predictions = algo.test(testset)

#### Recommend 5 Restaurants to users

In [ ]:
user_1 = 'BPKpLbR9NuWFAR9SUWpZOw'
restaurants = review['business_id'].unique()
user_1_ratings = []
for restaurant in restaurants:
    user_1_ratings.append((restaurant, algo.predict(user_1, restaurant).est))

# sort the recommended restaurants by predicted rating
sorted_ratings = sorted(user_1_ratings, key=lambda x: x[1], reverse=True)

# print the top 5 recommended restaurants
for i in range(5):
    business_id = sorted_ratings[i][0]
    name = review.loc[review['business_id'] == business_id, 'name'].iloc[0]
    print(f'{i+1}. {name}')

1. "Culinary Dropout"
2. "Rancho de Tia Rosa"
3. "Oscar's Steakhouse"
4. "Cantina Laredo"
5. "Pita Jungle"


In [ ]:
accuracy.mae(predictions)
accuracy.mse(predictions)
accuracy.rmse(predictions)
accuracy.fcp(predictions)

MAE:  0.0357
MSE: 0.0038
RMSE: 0.0615
FCP:  0.5415


0.5414867298796749

### 5. KNN Basic

In [ ]:
parameters={
    'name':['cosine','pearson'],
    'min_k': [3, 6, 9],
    'min_support': [True, False],
    'user_based':[True, False]
}

In [ ]:
grid=GridSearchCV(KNNBasic, param_grid=parameters,n_jobs=-1)
grid.fit(data)

In [ ]:
algo = BaselineOnly(bsl_options=bsl_options)

predictions = algo.fit(trainset).test(testset)
accuracy.mae(predictions)
accuracy.mse(predictions)
accuracy.rmse(predictions)
accuracy.fcp(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
MAE:  0.0362
MSE: 0.0040
RMSE: 0.0636
FCP:  0.5582


0.5581521594684385

In [ ]:
print(grid.best_score)
print(grid.best_params)

{'rmse': 0.08790605647149062, 'mae': 0.05774443005380259}
{'rmse': {'name': 'cosine', 'min_k': 6, 'min_support': True, 'user_based': True}, 'mae': {'name': 'cosine', 'min_k': 3, 'min_support': True, 'user_based': True}}


In [ ]:
algo = grid.best_estimator["mae"]
algo.fit(data.build_full_trainset())

Computing the msd similarity matrix...
Done computing similarity matrix.


#### Make recommendations

In [ ]:
predictions = algo.test(testset)

#### Recommend 5 Restaurants to users

In [ ]:
user_1 = 'BPKpLbR9NuWFAR9SUWpZOw'
restaurants = review['business_id'].unique()
user_1_ratings = []
for restaurant in restaurants:
    user_1_ratings.append((restaurant, algo.predict(user_1, restaurant).est))

# sort the recommended restaurants by predicted rating
sorted_ratings = sorted(user_1_ratings, key=lambda x: x[1], reverse=True)

# print the top 5 recommended restaurants
for i in range(5):
    business_id = sorted_ratings[i][0]
    name = review.loc[review['business_id'] == business_id, 'name'].iloc[0]
    print(f'{i+1}. {name}')

1. "Craig's Cake Shop"
2. "Papa Chevo's Taco Shop"
3. "Barro's Pizza"
4. "Eiffel Tower"
5. "Nottingham Inn Kitchen & Creamery"


In [ ]:
accuracy.mae(predictions)
accuracy.mse(predictions)
accuracy.rmse(predictions)
accuracy.fcp(predictions)

MAE:  0.0484
MSE: 0.0062
RMSE: 0.0790
FCP:  0.5175


0.5175494334256191

### Comparision of KNN Algorithms

In [ ]:
import plotly.graph_objects as go
knn_basic_scores = [0.0484, 0.0062, 0.0790, 0.5175]
knn_baseline_scores = [0.0464, 0.0055, 0.0745, 0.5520]
knn_zscore_scores = [0.0357, 0.0038, 0.5415, 0.5415]
knn_means_scores = [0.0278, 0.0024, 0.0492, 0.5522]

metrics = ['MAE', 'MSE', 'RMSE', 'FCP']
algorithms = ['KNN Basic', 'KNN Baseline', 'KNN with Z-Score', 'KNN with Means']

data = [
    go.Heatmap(
        x=metrics,
        y=algorithms,
        z=[knn_basic_scores, knn_baseline_scores, knn_zscore_scores, knn_means_scores],
        colorscale='Viridis'
    )
]

layout = go.Layout(
    title='Performance Comparison of KNN Algorithms',
    xaxis=dict(title='Evaluation Metric'),
    yaxis=dict(title='KNN Algorithm')
)

fig = go.Figure(data=data, layout=layout)

fig.show()

In [ ]:
knn_basic_data = go.Scatter(x=metrics, y=knn_basic_scores, mode='lines+markers', name='KNN Basic')
knn_baseline_data = go.Scatter(x=metrics, y=knn_baseline_scores, mode='lines+markers', name='KNN Baseline')
knn_zscore_data = go.Scatter(x=metrics, y=knn_zscore_scores, mode='lines+markers', name='KNN with Z-Score')
knn_means_data = go.Scatter(x=metrics, y=knn_means_scores, mode='lines+markers', name='KNN with Means')

# Define the layout for the line graph
layout = go.Layout(
    title='Performance Comparison of KNN Algorithms',
    xaxis=dict(title='Evaluation Metric'),
    yaxis=dict(title='Mean Score')
)

# Combine the line data and layout into a Plotly figure
fig = go.Figure(data=[knn_basic_data, knn_baseline_data, knn_zscore_data, knn_means_data], layout=layout)

# Show the line graph
fig.show()

## 2. NMF

### 1. SVD

In [ ]:
parameters={
    'n_factors':[20,50,80],
    'reg_all': [0.04, 0.06, 0.10],
    'n_epochs': [10, 20, 30, 50, 100],
    'lr_all':[0.002, 0.005, 0.01]
}

In [ ]:
grid=GridSearchCV(SVD, param_grid=parameters,n_jobs=-1)
grid.fit(data)

In [ ]:
print(grid.best_score)
print(grid.best_params)

{'rmse': 0.08918589319325369, 'mae': 0.056935822467182096}
{'rmse': {'n_factors': 20, 'reg_all': 0.1, 'n_epochs': 100, 'lr_all': 0.002}, 'mae': {'n_factors': 20, 'reg_all': 0.1, 'n_epochs': 50, 'lr_all': 0.01}}


In [ ]:
algo = grid.best_estimator["rmse"]
algo.fit(data.build_full_trainset())

#### Make recommendations

In [ ]:
predictions = algo.test(testset)

#### Recommend 5 Restaurants to users

In [ ]:
user_1 = 'BPKpLbR9NuWFAR9SUWpZOw'
restaurants = review['business_id'].unique()
user_1_ratings = []
for restaurant in restaurants:
    user_1_ratings.append((restaurant, algo.predict(user_1, restaurant).est))

# sort the recommended restaurants by predicted rating
sorted_ratings = sorted(user_1_ratings, key=lambda x: x[1], reverse=True)

# print the top 5 recommended restaurants
for i in range(5):
    business_id = sorted_ratings[i][0]
    name = review.loc[review['business_id'] == business_id, 'name'].iloc[0]
    print(f'{i+1}. {name}')

1. "Papa Chevo's Taco Shop"
2. "Macayo's Mexican Table"
3. "Kona Grill"
4. "Joyride Taco House"
5. "Original Breakfast House"


In [ ]:
accuracy.mae(predictions)
accuracy.mse(predictions)
accuracy.rmse(predictions)
accuracy.fcp(predictions)

MAE:  0.0439
MSE: 0.0045
RMSE: 0.0670
FCP:  0.5509


0.5509170379259515

### 2. SVDPP

##### 1. Using Implicit Feeback (Checkins)

In [ ]:
checkin_df = pd.read_csv("drive/MyDrive/DIC/yelp_checkin.csv")

In [ ]:
checkin_df = checkin_df.drop(['weekday','hour'], axis=1)
checkin_df

In [ ]:
checkin_df = checkin_df.groupby('business_id')['checkins'].sum()

In [ ]:
business_checkin_df = pd.merge(review, checkin_df, on='business_id', how='left')

In [ ]:
business_checkin_df.fillna(0, inplace=True)
business_checkin_df
print(business_checkin_df['checkins'].max())
print(business_checkin_df['checkins'].min())

32393.0
0.0


In [ ]:
reader_checkin = Reader(rating_scale=(0, 32393))
data_checkins = Dataset.load_from_df(business_checkin_df[['user_id', 'business_id', 'checkins']], reader_checkin)
trainset_checkin, testset_checkin = train_test_split(data, test_size=0.2)
algo = SVDpp()
algo.fit(trainset_checkin)

###### Make recommendations

In [ ]:
predictions = algo.test(testset_checkin)

NameError: ignored

###### Recommend 5 Restaurants to users

In [ ]:
user_1 = 'BPKpLbR9NuWFAR9SUWpZOw'
restaurants = review['business_id'].unique()
user_1_ratings = []
for restaurant in restaurants:
    user_1_ratings.append((restaurant, algo.predict(user_1, restaurant).est))

# sort the recommended restaurants by predicted rating
sorted_ratings = sorted(user_1_ratings, key=lambda x: x[1], reverse=True)

# print the top 5 recommended restaurants
for i in range(5):
    business_id = sorted_ratings[i][0]
    name = review.loc[review['business_id'] == business_id, 'name'].iloc[0]
    print(f'{i+1}. {name}')

1. "Eiffel Tower"
2. "McDonald's"
3. "Crab Corner"
4. "Aces Bar & Grill"
5. "Carmine's Pizza Kitchen"


In [ ]:
accuracy.mae(predictions)
accuracy.mse(predictions)
accuracy.rmse(predictions)
accuracy.fcp(predictions)

MAE:  0.0612
MSE: 0.0086
RMSE: 0.0929
FCP:  0.4791


0.47906405953531334

##### 2. Using Explicit Feedback (Ratings)

###### Make recommendations

In [ ]:
parameters={
    'n_factors':[20,50],
    'reg_all': [0.04, 0.06],
    'n_epochs': [10, 20],
    'lr_all':[0.002, 0.005]
}

In [ ]:
grid=GridSearchCV(SVDpp, param_grid=parameters,n_jobs=-1)
grid.fit(data)

In [ ]:
print(grid.best_score)
print(grid.best_params)

{'rmse': 0.09188648675557219, 'mae': 0.059843975031419916}
{'rmse': {'n_factors': 20, 'reg_all': 0.06, 'n_epochs': 20, 'lr_all': 0.005}, 'mae': {'n_factors': 20, 'reg_all': 0.06, 'n_epochs': 20, 'lr_all': 0.005}}


In [ ]:
algo = grid.best_estimator["rmse"]
algo.fit(data.build_full_trainset())

In [ ]:
predictions = algo.test(testset)

###### Recommend 5 Restaurants to users

In [ ]:
user_1 = 'BPKpLbR9NuWFAR9SUWpZOw'
restaurants = review['business_id'].unique()
user_1_ratings = []
for restaurant in restaurants:
    user_1_ratings.append((restaurant, algo.predict(user_1, restaurant).est))

# sort the recommended restaurants by predicted rating
sorted_ratings = sorted(user_1_ratings, key=lambda x: x[1], reverse=True)

# print the top 5 recommended restaurants
for i in range(5):
    business_id = sorted_ratings[i][0]
    name = review.loc[review['business_id'] == business_id, 'name'].iloc[0]
    print(f'{i+1}. {name}')

1. "Eiffel Tower"
2. "Walmart Supercenter"
3. "Bachi Burger"
4. "Ocean One Bar and Grille"
5. "Charlies  Lakeside Restaurant & Lounge"


In [ ]:
accuracy.mae(predictions)
accuracy.mse(predictions)
accuracy.rmse(predictions)
accuracy.fcp(predictions)

MAE:  0.0506
MSE: 0.0059
RMSE: 0.0766
FCP:  0.5359


0.535913348883918

### 3. NMF

In [ ]:
parameters={
    'n_factors':[20,50, 80],
    'n_epochs': [10, 20, 30]
}

In [ ]:
grid=GridSearchCV(NMF, param_grid=parameters,n_jobs=-1)
grid.fit(data)

In [ ]:
print(grid.best_score)
print(grid.best_params)

{'rmse': 0.09545103974628169, 'mae': 0.05320359396420875}
{'rmse': {'n_factors': 50, 'n_epochs': 10}, 'mae': {'n_factors': 80, 'n_epochs': 10}}


In [ ]:
algo = grid.best_estimator["rmse"]
algo.fit(data.build_full_trainset())

In [ ]:
algo = NMF()
algo.fit(trainset)

#### Make recommendations

In [ ]:
predictions = algo.test(testset)

#### Recommend 5 Restaurants to users

In [ ]:
user_1 = 'BPKpLbR9NuWFAR9SUWpZOw'
restaurants = review['business_id'].unique()
user_1_ratings = []
for restaurant in restaurants:
    user_1_ratings.append((restaurant, algo.predict(user_1, restaurant).est))

# sort the recommended restaurants by predicted rating
sorted_ratings = sorted(user_1_ratings, key=lambda x: x[1], reverse=True)

# print the top 5 recommended restaurants
for i in range(5):
    business_id = sorted_ratings[i][0]
    name = review.loc[review['business_id'] == business_id, 'name'].iloc[0]
    print(f'{i+1}. {name}')

1. "Craig's Cake Shop"
2. "Papa Chevo's Taco Shop"
3. "Barro's Pizza"
4. "Eiffel Tower"
5. "Nottingham Inn Kitchen & Creamery"


In [ ]:
accuracy.mae(predictions)
accuracy.mse(predictions)
accuracy.rmse(predictions)
accuracy.fcp(predictions)

MAE:  0.0452
MSE: 0.0072
RMSE: 0.0851
FCP:  0.2291


0.22907262500899717

### Comparing Performance of NMF techniques

In [ ]:
import plotly.graph_objects as go

algorithm_names = ['SVD', 'SVDpp', 'NMF']
mae_values = [0.0439, 0.0506, 0.0452]
mse_values = [0.0045, 0.0059, 0.0072]
rmse_values = [0.0670, 0.0766, 0.0851]
fcp_values = [0.5509, 0.5359, 0.2291]

fig = go.Figure()
fig.add_trace(go.Bar(x=algorithm_names, y=rmse_values, name='RMSE'))
fig.add_trace(go.Bar(x=algorithm_names, y=mse_values, name='MSE'))
fig.add_trace(go.Bar(x=algorithm_names, y=mae_values, name='MAE'))
fig.add_trace(go.Bar(x=algorithm_names, y=fcp_values, name='FCP'))

fig.update_layout(
    title='Comparison of evaluation metrics for different algorithms',
    xaxis_title='Algorithm',
    yaxis_title='Value')

fig.show()

## 3. BaselineOnly

In [ ]:
parameters = {'bsl_options': {'method': ['als', 'sgd'],
                              'reg_u': [10, 20, 50],
                              'reg_i': [5, 10, 15],
                              'n_epochs': [100, 150, 200],
                              'learning_rate': [0.001, 0.01, 0.1]}}

In [ ]:
grid=GridSearchCV(BaselineOnly, param_grid=parameters,n_jobs=-1)
grid.fit(data)

In [ ]:
print(grid.best_score)
print(grid.best_params)

{'rmse': 0.08573673204517065, 'mae': 0.05358712972024675}
{'rmse': {'bsl_options': {'method': 'als', 'reg_u': 10, 'reg_i': 15, 'n_epochs': 100, 'learning_rate': 0.001}}, 'mae': {'bsl_options': {'method': 'sgd', 'reg_u': 10, 'reg_i': 5, 'n_epochs': 100, 'learning_rate': 0.001}}}


In [ ]:
algo = grid.best_estimator["rmse"]
algo.fit(data.build_full_trainset())

Estimating biases using als...


#### Make recommendations

In [ ]:
predictions = algo.test(testset)

#### Recommend 5 Restaurants to users

In [ ]:
user_1 = 'BPKpLbR9NuWFAR9SUWpZOw'
restaurants = review['business_id'].unique()
user_1_ratings = []
for restaurant in restaurants:
    user_1_ratings.append((restaurant, algo.predict(user_1, restaurant).est))

# sort the recommended restaurants by predicted rating
sorted_ratings = sorted(user_1_ratings, key=lambda x: x[1], reverse=True)

# print the top 5 recommended restaurants
for i in range(5):
    business_id = sorted_ratings[i][0]
    name = review.loc[review['business_id'] == business_id, 'name'].iloc[0]
    print(f'{i+1}. {name}')

1. "Oyster Bar"
2. "CUT by Wolfgang Puck"
3. "Mesa Grill"
4. "Scarpetta"
5. "Yardbird Southern Table & Bar"


In [ ]:
accuracy.mae(predictions)
accuracy.mse(predictions)
accuracy.rmse(predictions)
accuracy.fcp(predictions)

MAE:  0.0480
MSE: 0.0057
RMSE: 0.0755
FCP:  0.5610


0.5610393603936039

In [ ]:
bsl_options = {'method': 'als',
               'n_epochs': 50,
               'reg_u': 20,
               'reg_i': 10,
               'learning_rate':1
               }
algo = BaselineOnly(bsl_options=bsl_options)
cross_validate(algo, data, measures=['RMSE', 'FCP', 'MSE', 'MAE'], cv=1, verbose=False)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([0.08762663, 0.08646312, 0.08458428, 0.08684498, 0.08558994]),
 'test_fcp': array([0.50637668, 0.50883664, 0.53823699, 0.51156001, 0.51329518]),
 'test_mse': array([0.00767843, 0.00747587, 0.0071545 , 0.00754205, 0.00732564]),
 'test_mae': array([0.05516241, 0.05431599, 0.05361455, 0.0548308 , 0.05387947]),
 'fit_time': (2.4071576595306396,
  1.083712100982666,
  1.1120872497558594,
  1.5982182025909424,
  1.2370963096618652),
 'test_time': (0.05538582801818848,
  0.053986549377441406,
  0.05571913719177246,
  0.4529423713684082,
  0.05033612251281738)}

## Comparing all Collaborative Filtering Models

In [10]:
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots

# Create the data
data = {'test_rmse': [0.0928, 0.0755, 0.08692292],
        'test_mae': [0.0567, 0.0480, 0.05766862],
        'test_fcp': [0.5609, 0.5379, 0.63493503],
        'fit_time': [1.8923487663269043, 1.9373219013214111, 2.66087007522583],
        'test_time': [0.11574029922485352, 0.0867300033569336, 0.2719612121582031]}
index = ['SVD', 'BaselineOnly', 'KNNBasic']
df = pd.DataFrame(data=data, index=index)

# Create the subplots
fig = make_subplots(rows=2, cols=2, subplot_titles=('test_rmse', 'test_mae', 'test_fcp', 'fit_time'))

fig.add_trace(go.Bar(x=df.index, y=df['test_rmse'], name='RMSE'), row=1, col=1)
fig.add_trace(go.Bar(x=df.index, y=df['test_mae'], name='MAE'), row=1, col=2)
fig.add_trace(go.Bar(x=df.index, y=df['test_fcp'], name='FCP'), row=2, col=1)
fig.add_trace(go.Bar(x=df.index, y=df['fit_time'], name='Fit Time'), row=2, col=2)

fig.update_layout(height=600, width=800, title_text="Performance Metrics by Algorithm")
fig.show()

## References - 

https://towardsdatascience.com/building-and-testing-recommender-systems-with-surprise-step-by-step-d4ba702ef80b
https://surprise.readthedocs.io/